This code generates the model SNe for all of the models. Check up on Model 4, I think the mixed map has a normalization error. (map built in 3.4cModelVisualization)

In [2]:
#import packages and initialize settings
import time
import os
import numpy as np
from astropy.io import fits, ascii
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table


plt.rc('text',usetex=True)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

import sys
# Work Laptop:     
#sys.path.append('/Users/mayker.1/Desktop/NessPythonFunctions')
# Work Desktop:    
sys.path.append('/home/mayker.1/Desktop/PythonFunctions')
# Personal Laptop: sys.path.append('/Users/nessmayker/Desktop/NessPythonFunctions')
from FindNearestMC import nearestMCMethodSigMol
from SNeGenerationModels import runModels

In [3]:
#Get Galaxy names, distances, and fits file locations

# Unpack master table to get SNe information
database = Table.read('../Data/3.SNe+GalData.csv', format = "csv")
centerCoords = [(database["orient_ra"][i], database["orient_dec"][i]) for i in range(len(database))]   

database[0:1]



galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient,ImageFile150,ErrFile150,AlphaCOFile,PACovFile,IFRFile,MixedMaps,IFRResizedMaps,MassMaps
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str112,str113,str80,str112,str97,str37,str54,str35
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/ESO097-013_alphaCO21_scaling.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_coverage2d.fits,/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/z0mgs/eso097-013_w1_gauss7p5.fits,../Data/MixedMaps/circinus_Mixed.fits,../Data/ReprojectedInfraredMaps/circinus_reshaped.fits,../Data/MassMaps/circinus_Mass.fits


In [4]:

#Make lists unique for galaxies (no repeating galaxies or other data)    
def unique(origGalsList, origDistList, imageFileList, errFileList, centerCoords, posAngList, inclList, covFileList, ifrFileList, acoFileList, mixedFileList): 
    gals, dists, images, errFiles, coords, posAngs, inclinations, covFiles, ifrFiles, acoFiles, mixedFiles = [],[],[],[],[],[],[],[],[],[],[]
    for i in range(len(origGalsList)): 
        if origGalsList[i] not in gals: 
            if origGalsList[i] != "ngc1068" and origGalsList[i] != "ngc1672" and origGalsList[i] != "ngc4579":
                gals.append(origGalsList[i]) 
                dists.append(origDistList[i])
                images.append(imageFileList[i])
                errFiles.append(errFileList[i])
                coords.append(centerCoords[i])
                posAngs.append(posAngList[i])
                if inclList[i] == 90.0:
                    inclList[i] = 80.0
                inclinations.append(inclList[i]) 
                covFiles.append(covFileList[i])
                ifrFiles.append(ifrFileList[i])
                acoFiles.append(acoFileList[i])
                mixedFiles.append(mixedFileList[i])
            else:
                pass
    return(gals, dists, images, errFiles, coords, posAngs, inclinations, covFiles, ifrFiles, acoFiles, mixedFiles)

gals, dists, images, errFiles, coords, posAngs, inclinations, covFiles, ifrFiles, acoFiles, mixedFiles = unique(database["galaxy"], database["dist"],database["ImageFile150"], database["ErrFile150"], centerCoords, database["orient_posang"], database["orient_incl"], database["PACovFile"], database["IFRResizedMaps"], database["AlphaCOFile"], database["MixedMaps"])
print(gals)


['circinus', 'ngc0253', 'ngc0628', 'ngc1087', 'ngc1097', 'ngc1365', 'ngc1433', 'ngc1511', 'ngc1559', 'ngc1566', 'ngc1637', 'ngc2775', 'ngc2997', 'ngc3239', 'ngc3627', 'ngc4254', 'ngc4303', 'ngc4321', 'ngc4424', 'ngc4457', 'ngc4496a', 'ngc4536', 'ngc4945', 'ngc5128', 'ngc5236', 'ngc5530', 'ngc6744', 'ngc7793']


In [7]:
    
# run models 1-4 and store results
galCol, counter, n55mod1, n55mod2, n55mod3 = [],[],[],[],[]

for i in range(len(gals)):
    
    ifr_hdu = pyfits.open(ifrFiles[i])
    ifr_map = ifr_hdu[0].data
    cov_hdu = pyfits.open(covFiles[i])
    cov_map = cov_hdu[0].data

    wcs     = WCS(cov_hdu[0].header, naxis=2)
    naxis   = wcs._naxis # size of image naxis[0] = x and [1] = y
    grid    = np.indices((naxis[1],naxis[0]))
    ra, dec = wcs.wcs_pix2world(grid[1],grid[0],0) 

    f_cov = cov_map.flatten()
    f_ifr = ifr_map.flatten() 
    f_ra  = ra.flatten()
    f_dec = dec.flatten()

    keep = np.where((f_cov > 0.9) & np.isfinite(f_ifr))
    ifr  = f_ifr[keep]
    star_ra   = f_ra[keep]
    star_dec  = f_dec[keep]
    
    
    raArr1, decArr1, dxArr1, dyArr1 = runModels(gals[i], images[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=1)
    raArr2, decArr2, dxArr2, dyArr2 = runModels(gals[i], images[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=2)
    raArr3, decArr3, dxArr3, dyArr3 = runModels(gals[i], images[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=3, starLight = ifr, starRa = star_ra, starDec = star_dec)
    #raArr4, decArr4, dxArr4, dyArr4 = runModels(gals[i], mixedFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=2)

    n55_mod1 = nearestMCMethodSigMol(gals[i], images[i], errFiles[i], acoFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr1, decArr1)
    n55_mod2 = nearestMCMethodSigMol(gals[i], images[i], errFiles[i], acoFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr2, decArr2)
    n55_mod3 = nearestMCMethodSigMol(gals[i], images[i], errFiles[i], acoFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr3, decArr3)
    #n55_mod4, n65_mod4 = nearestMCMethod(gals[i], images[i], errFiles[i], massFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr4, decArr4)
    
    if (type(n55_mod1) != float and type(n55_mod1) != np.float64):
        for j in range(len(n55_mod1)):
            galCol.append(gals[i])
            counter.append(j)
            n55mod1.append(n55_mod1[j])
            n55mod2.append(n55_mod2[j])
            n55mod3.append(n55_mod3[j])
            #n55mod4.append(n55_mod4[j])
    else:
        for j in range(100):
            galCol.append(gals[i])
            counter.append(j)
            n55mod1.append(np.float('nan'))
            n55mod2.append(np.float('nan'))
            n55mod3.append(np.float('nan'))
            #n55mod4.append(np.float('nan'))              
    
results = Table({"Galaxy": galCol,
                "ModelNum": counter,
                "n55Mod1": n55mod1,
                "n55Mod2": n55mod2,
                "n55Mod3": n55mod3})

results.write("../Data/AllModels.csv", format = "csv", overwrite = True)

        
        
    

at SigMol cutoff A for circinus
Pixels with SigMol > 7.9 5648
Nearest sigmol [   0.            0.            0.          147.77464993  628.13924282
    0.            0.          130.04091541 1547.90809393  129.91375481
  828.56378562  120.07894427  242.59196722    0.         2827.95558438
  345.18288007    0.         1927.28266095    0.          482.16453596
    0.         1786.0240724   955.04459484 1582.62088256   60.05309083
    0.          120.00046724  362.80241882   60.06583319  461.01947298
 1496.49837862 1354.0680977   957.2414065     0.          203.7371784
  180.31133043  320.28021865  194.13630445    0.            0.
  572.51172899   60.0242447   717.68536271  147.70884377 1570.20533596
    0.            0.          124.67486244    0.            0.
    0.            0.            0.            0.            0.
  194.93099503  498.1801747  1971.27567445    0.          812.43964917
  537.15452681  726.65903248    0.            0.            0.
    0.            0.          242

at SigMol cutoff A for ngc0628
Pixels with SigMol > 7.9 622960
Nearest sigmol [180.39374476  30.17243423   0.         264.61677636   0.
   0.         347.68713541   0.          61.07304711   0.
  88.28842852   0.         736.86392484   0.          66.89018753
 209.54529307  57.85492524   0.         144.66086639 193.28130482
  47.77873946   0.          78.99031511   0.           0.
  76.94657781   0.         354.34722478 290.09274302   0.
   0.           0.           0.           0.         405.95021519
   0.         139.52697221 144.47437342   0.           0.
   0.          86.77144273 326.53441522   0.           0.
   0.           0.           0.           0.           0.
   0.          19.28859863 105.113066     0.           0.
   0.         240.94811609 164.22912365 196.6951141    9.55575117
 289.36342776 183.19839365 120.17535268   0.           0.
   0.           0.           0.         112.46044559   0.
  28.92741306   0.           0.         225.60509253  96.44840382
   0.       

at SigMol cutoff A for ngc1097
Pixels with SigMol > 7.9 43073
Nearest sigmol [1111.25772249  447.78369918  354.12119833  128.05490142    0.
  256.12548212   95.90600406   74.10112355  309.13526566   38.51688264
 1925.63900991  256.07082153    0.          447.79970848   85.26709559
   38.52688685  428.07593695  354.69717312  205.29673725  928.80242659
  102.6492877     0.          175.73631547   38.50933733   47.9524516
 1265.09162602  391.8647879    91.16106635  563.19798354   38.51750715
   38.51097651 2460.62508582  181.03663016   38.52540169    0.
    0.          613.60897934 1610.44045147  488.81055681  808.19095101
    0.          256.76334735  107.91698902  680.62962638 1371.50629169
  130.62099675  224.61925052  323.72630749  215.77819763  102.65378262
  961.31270155    0.         1892.48662924   38.53170342 1116.89326097
    0.          832.07998117    0.          385.49488341    0.
    0.          323.69508431  561.51668022  895.63698159   85.26968796
    0.            0.     

at SigMol cutoff A for ngc1433
Pixels with SigMol > 7.9 35392
Nearest sigmol [ 194.23425497  591.35759597  408.18613257 2001.78124275  530.38432336
    0.         1433.7708209   415.9023918    27.55074916   82.64881497
  881.7606051    60.72017045  236.49130791   60.90043082    0.
  901.86730863 1019.6887223     0.          267.09002902 1640.92741871
  244.04631386   55.09855074  826.83628948    0.          606.08604295
  373.664282    880.96869415  182.78750642  317.79158989  398.81629335
    0.          805.62392778   39.3402475   581.52696104  213.25806686
  403.66666956  722.25677938 1290.53021542 2410.92455177  141.44964105
   60.71413812  441.51946774  513.70170977  636.90278882   68.89559146
  360.98935676    0.          165.29781163 1123.15230155 1046.92861742
   97.57032325  170.89163309  258.83613127 1017.55932787   60.72142362
  630.49715926  570.29645582 1383.91721778 1154.51303261  730.24116147
 1058.51632845    0.         1044.32211011  274.45391499  128.1655584
   55.097

at SigMol cutoff A for ngc1559
Pixels with SigMol > 7.9 52919
Nearest sigmol [7404.91920999  166.74761436 4225.36565178  630.23253117    0.
    0.          370.04702085 1458.09724748  125.96741436   38.80965204
 1689.09670249 1439.27194116 3716.88111493 2766.77895972 4295.86738015
 2627.44877349 3661.24662242 2100.35307267    0.         6288.96602902
  830.78671738   92.51016525    0.            0.         2166.37603682
    0.          126.54881653 1824.90635352  253.08764812  296.97697602
   38.81283943  310.33138694   77.63371015 3011.31235814  250.07149707
 6005.64046604   98.99177499    0.         3159.77640817   63.27381704
 2766.76029481 1659.71124561  523.10767602 3860.19477322  701.85868066
 2573.33728399 2834.3755895  1237.40773758  640.46685732 5242.3088436
 2701.53359024 3842.23589935  469.34625981    0.          253.05490667
 5816.10500472    0.         1858.73767868  698.63871822   92.6302895
  584.88034114  250.89882627  168.39711832 1522.99104131  228.34832183
  126.5892

...

at SigMol cutoff A for ngc4496a
Pixels with SigMol > 7.9 10036
Nearest sigmol [1499.51412315   75.74323679    0.          157.0475017     0.
  557.07245282  115.43944779  446.67332395  158.35574179  283.86692921
 1340.15312933  388.95399147  111.1292073   201.01148568  322.33467797
  394.76724242  370.62924501 1119.08230928  233.08733623 1193.85980475
  354.65516867    0.          225.6785142    84.92021232    0.
   50.34447144  649.48760109   61.77153233  306.52301455  383.73986515
  377.39389207  692.6343123   139.26883384  344.79044366  100.68897805
  227.82737857  252.38647369  234.31858384    0.          285.05822135
  322.41349555   28.47806528  306.51877311    0.            0.
    0.          261.76210188  291.56007033  834.39460419  191.44107025
    0.          665.75244438  139.26916609   61.77181343  370.21024335
   77.09706907    0.         1069.26183723    0.          339.68184524
  683.98260464  166.6944965   335.82431018    0.            0.
   50.34500627  194.62678457  5

/home/mayker.1/Desktop/PythonFunctions/FindNearestMC.py:186: RuntimeWarning: invalid value encountered in multiply
  sigmol = aco * inten * np.cos(incl * np.pi/180.0)


at SigMol cutoff A for ngc4945
Pixels with SigMol > 7.9 20691
Nearest sigmol [ 810.90010656    0.            0.         1695.31367917    0.
  636.23283396  828.66002384    0.          612.38879947    0.
    0.            0.            0.            0.         2141.28977418
 1136.49818235   48.04800228    0.          426.22171936    0.
  956.48249882  484.1982667   786.54462     191.58920692  582.21879863
    0.          598.57005355  414.51372805 1057.33489355  321.85043391
    0.         2352.82980516    0.         1242.42706537    0.
    0.            0.         1137.14622861    0.         1993.51182416
  335.19810686  316.39139517  721.03033915  526.73081142  624.35953403
  908.02729896  136.60474603  478.88580856    0.          143.35402332
    0.          191.59854776  409.03068109    0.         1385.90480727
  273.737322   1538.43633259 1016.13557751    0.          705.36656609
   48.04759877    0.          576.68790531  819.65479672    0.
    0.         1095.04965989    0.      

at SigMol cutoff A for ngc5236
Pixels with SigMol > 7.9 540621
Nearest sigmol [   0.            0.            0.           37.28052134   42.90097201
    0.           18.35467702    0.            0.            0.
    0.            0.            0.           69.00761771    0.
   28.77418086    0.           53.53666739  137.84389364    0.
  324.46830417    0.            0.          100.57679269  334.50100068
    0.            0.            0.            0.           37.28081722
  210.4737559   358.96056756    0.            0.           74.47303136
    0.            0.            0.            0.            0.
  214.52919111    0.            0.           68.9550207     0.
    0.          230.96280232  352.83374549    0.          250.87481005
    0.            0.          503.28653569  126.00088583    0.
    0.           12.42696251   59.39311148   53.52795061  321.90001191
    0.            0.          368.38003678   85.79002427   64.46139007
    0.          840.90266626    0.            0

at SigMol cutoff A for ngc6744
Pixels with SigMol > 7.9 151981
Nearest sigmol [177.0882107  336.03446846 116.45248147 361.86085364 105.63576245
 354.81768218 592.56432004   0.         186.0913569    0.
 241.21484793 236.64302697 136.2195459    0.           0.
   0.         323.85741057   0.         153.24294719   0.
 180.90704913 283.32503122   0.          56.30585885 240.76611741
 145.95226117 392.70030543  14.33025132 106.24863832 343.96829935
   0.           0.         171.55341842 150.60588395  78.38470818
 206.97128752   0.          97.42142339   0.         263.56579658
  52.8348277  541.27248167 731.8785833  520.85655099 355.47050489
 335.16833457  23.31143985   0.         208.86900544   0.
   0.          86.22618023  74.70368393 215.55982517  69.98011742
   0.         194.87608118 230.07038266 180.65691938  42.98165129
  68.63398484   0.          40.13234549   0.          43.4096249
   0.         185.98985317   0.           0.         248.0675682
 721.73175754 344.59410847 118.8